<a href="https://colab.research.google.com/github/generalMG/MusicVAE/blob/main/MusicVAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import ALL required packages and libraries

In [ ]:
import glob
BASE_DIR = "gs://download.magenta.tensorflow.org/models/music_vae/colab2"
print("Dependencies installation...")
!apt-get update -qq && apt-get install -qq libfluidsynth1 fluid-soundfont-gm build-essential libasound2-dev libjack-dev
!pip install -q pyfluidsynth
!pip install -U -q magenta
!gsutil -q -m cp gs://magentadata/models/music_vae/groovae_colab/*wav .
!gsutil -q -m cp gs://magentadata/models/music_vae/checkpoints/*.tar .
!gsutil -q -m cp -R gs://download.magenta.tensorflow.org/models/music_vae/colab2/checkpoints/mel_2bar_big.ckpt.* /content/


In [2]:
import ctypes.util
orig_ctypes_util_find_library = ctypes.util.find_library
def proxy_find_library(lib):
  if lib == 'fluidsynth':
    return 'libfluidsynth.so.1'
  else:
    return orig_ctypes_util_find_library(lib)

ctypes.util.find_library = proxy_find_library

In [ ]:
from google.colab import files
import magenta
import magenta.music as mm
from magenta.models.music_vae import configs
from magenta.models.music_vae.trained_model import TrainedModel
import numpy as np
import os
import note_seq
import tensorflow
print(magenta.__version__)
print(tensorflow.__version__)
import tensorflow_datasets as tfds
def quantize(s, steps_per_quarter=4):
  return note_seq.sequences_lib.quantize_note_sequence(s,steps_per_quarter)
def set_to_drums(ns):
  for n in ns.notes:
    n.instrument=9
    n.is_drum = True

def is_4_4(s):
  ts = s.time_signatures[0]
  return (ts.numerator == 4 and ts.denominator ==4)

dataset_2bar = tfds.as_numpy(tfds.load(
    name="groove/2bar-midionly",
    split=tfds.Split.VALIDATION,
    try_gcs=True))

dev_sequences = [quantize(note_seq.midi_to_note_sequence(features["midi"])) for features in dataset_2bar]
_ = [set_to_drums(s) for s in dev_sequences]
dev_sequences = [s for s in dev_sequences if is_4_4(s) and len(s.notes) > 0 and s.notes[-1].quantized_end_step > note_seq.steps_per_bar_in_quantized_sequence(s)]

dataset_4bar = tfds.as_numpy(tfds.load(
    name="groove/4bar-midionly",
    split=tfds.Split.VALIDATION,
    try_gcs=True))

dev_sequences_4bar = [quantize(note_seq.midi_to_note_sequence(features["midi"])) for features in dataset_4bar]
_ = [set_to_drums(s) for s in dev_sequences_4bar]
dev_sequences_4bar = [s for s in dev_sequences_4bar if is_4_4(s) and len(s.notes) > 0 and s.notes[-1].quantized_end_step > note_seq.steps_per_bar_in_quantized_sequence(s)]



In [4]:
def play(note_sequence):
  mm.play_sequence(note_sequence, synth=mm.fluidsynth)

def interpolate(model, start_seq, end_seq, num_steps, max_length=32,
                assert_same_length=True, temperature=0.5, individual_duration=4.0):
  note_sequences = model.interpolate(start_seq, end_seq, num_steps=num_steps, length=max_length, temperature=temperature, assert_same_length=assert_same_length)
  print('Start Seq Reconstruction')
  play(note_sequences[0])
  print('End Seq Reconstruction')
  play(note_sequences[-1])
  print('Mean Sequence')
  play(note_sequences[num_steps // 2])
  print('Start -> End Interpolation')
  interp_seq = mm.sequences_lib.concatenate_sequences(note_sequences, [individual_duration] * len(note_sequences))
  play(interp_seq)
  mm.plot_sequence(interp_seq)
  return interp_seq if num_steps > 3 else note_sequences[num_steps // 2]
def download(note_sequence, filename):
  mm.sequence_proto_to_midi_file(note_sequence, filename)
  files.download(filename)

In order to init. models click the run botton of the cell below.


In [ ]:
print("Initializing Music VAE...")
music_vae = {}
cat_mel_2bar_big = configs.CONFIG_MAP['cat-mel_2bar_big']
cat_drums_2bar_small = configs.CONFIG_MAP['cat-drums_2bar_small']
nade_drums_2bar_reduced = configs.CONFIG_MAP['nade-drums_2bar_reduced']
nade_drums_2bar_full = configs.CONFIG_MAP['nade-drums_2bar_full']
music_vae['cat_mel_2bar_big'] = TrainedModel(cat_mel_2bar_big, batch_size=4, checkpoint_dir_or_path='/content/mel_2bar_big.ckpt')
music_vae['drums_2bar_oh_lokl'] = TrainedModel(cat_drums_2bar_small, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/drums_2bar_small.lokl.ckpt')
music_vae['drums_2bar_oh_hikl'] = TrainedModel(cat_drums_2bar_small, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/drums_2bar_small.hikl.ckpt')
music_vae['drums_2bar_nade_reduced'] = TrainedModel(nade_drums_2bar_reduced, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/drums_2bar_nade.reduced.ckpt')
music_vae['drums_2bar_nade_full'] = TrainedModel(nade_drums_2bar_full, batch_size=4, checkpoint_dir_or_path=BASE_DIR + '/checkpoints/drums_2bar_nade.full.ckpt')


Generate Samples

In [ ]:
model = 'drums_2bar_oh_lokl' #@param ["cat_mel_2bar_big","drums_2bar_oh_lokl", "drums_2bar_oh_hikl", "drums_2bar_nade_reduced", "drums_2bar_nade_full"]
temperature = 0.5 #@param {type:"slider", min:0.1, max:2.0, step:0.1}
n = 2 #@param {type:"slider", min:2, max:10, step:1}
length = 10 #@param {type:"slider", min:5, max:80, step:1}
generated_sequences = music_vae[model].sample(n=n, length=length, temperature=temperature)
for i, ns in enumerate(generated_sequences):
  note_seq.plot_sequence(ns)
  note_seq.play_sequence(ns, synth=note_seq.fluidsynth)

Cell below downloads note sequence as MIDI files in your LOCAL SYSTEM


In [ ]:
#Download MIDI files to local system
for i, ns in enumerate(generated_sequences):
  download(ns, '%s_sample_%d.mid' % (model, i))

Please Upload saved MIDI files back again from Local System to COLAB\
NB: I was trying to directly implement MIDI files/note sequence to the model without back and forth downloading/uploading process, but I am lacking a bit deeper knowledge of the note sequence/midi

In [ ]:
# Upload saved MIDI files from Local System to COLAB
## NB: I was trying to directly implement MIDI files/note sequence to the model without back and forth downloading/uploading process, but I am lacking a bit deeper knowledge of the note sequence/midi
input_drums_midi_data = files.upload().values() or input_drums_midi_data

In [42]:
input_drums_midi_data = [
    tensorflow.io.gfile.GFile(fn, mode='rb').read()
    for fn in sorted(tensorflow.io.gfile.glob(BASE_DIR + '/midi/drums_2bar*.mid'))]

In [ ]:
# if all steps above were followed correctly there shouldn't be a problem with this particular step
drums_input_seqs = [mm.midi_to_sequence_proto(m) for m in input_drums_midi_data]
extracted_beats = []
# Config shall be changed according to the music_vae model
config = cat_drums_2bar_small # Change here from the cells above or copy paste: cat_mel_2bar_big, cat_drums_2bar_small, nade_drums_2bar_reduced, nade_drums_2bar_full
for ns in drums_input_seqs:
  extracted_beats.extend(config.data_converter.from_tensors(
      config.data_converter.to_tensors(ns)[1]))
for i, ns in enumerate(extracted_beats):
  print("Beat", i)
  #play(ns)
  note_seq.plot_sequence(ns)
  note_seq.play_sequence(ns, synth=note_seq.fluidsynth)

# Generate Interpolations

In [ ]:
#@title Interpolate between 2 beats, selected from those in the previous cell.
model = "drums_2bar_oh_lokl" #@param ["cat_mel_2bar_big","drums_2bar_oh_lokl", "drums_2bar_oh_hikl", "drums_2bar_nade_reduced", "drums_2bar_nade_full"]
start_beat = 0 #@param {type:"integer"}
end_beat = 1 #@param {type:"integer"}
start_beat = extracted_beats[start_beat]
end_beat = extracted_beats[end_beat]

temperature = 0.5 #@param {type:"slider", min:0.1, max:1.5, step:0.1}
num_steps = 13 #@param {type:"integer"}

drums_interp = interpolate(music_vae[model], start_beat, end_beat, num_steps=num_steps, temperature=temperature)